In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## ניקוי נתונים פיזיקליים וטיפול בטיפוסים (Cleaning & Type Handling)

בעת ניתוח נתונים מניסויים פיזיקליים, נתונים גולמיים עלולים לכלול שגיאות נפוצות:  
ערכים חסרים במדידות, יחידות מעורבות, רווחים מיותרים בשמות חומרים, טיפוסי נתונים שגויים (מספרים כטקסט),  
או שורות כפולות של מדידות זהות.  

שלבי הניקוי הראשוניים של **Pandas** מאפשרים להכין את הנתונים לעיבוד מדעי מדויק,  
להבטיח עקביות בין יחידות מידה, ולמנוע טעויות חישוב בהמשך.

#### טיפול בערכים חסרים: `isna`, `fillna`, `dropna`

- `isna()` / `notna()` — לזיהוי מדידות חסרות (NaN או None), למשל ערכי טמפרטורה שלא נרשמו.  
- `fillna()` — למילוי ערכים חסרים בערך ממוצע, מדידה סמוכה (forward/backward fill), או ערך ברירת מחדל.  
  לדוגמה, ניתן למלא ערך אנרגיה חסר לפי ממוצע מדידות קודמות.  
- `dropna()` — להסרת שורות או עמודות שבהן חסרים נתונים קריטיים, כגון תוצאות מדידה לא שלמות.

#### מחרוזות, קטגוריות ומספרים: ניקוי ותקנון נתונים

- גישת המחרוזות `str` מאפשרת פעולות טקסט על שמות מדגמים או חומרים:  
  `str.strip()` להסרת רווחים, `str.lower()` לאחידות כתיב, `str.replace()` לתיקון סימונים שגויים.  
- המרת טיפוסים:
  - `astype('category')` — להפיכת שמות ניסויים או סוגי חומרים לקטגוריות.  
  - `pd.to_numeric()` — להמרת נתונים נומריים (כמו אנרגיה, טמפרטורה, שדה מגנטי) ממחרוזות לערכים מספריים אמיתיים.  
  - `pd.to_datetime()` — להמרת מחרוזות של תאריכים לפורמט זמן מדעי מדויק.  
- פירוק מחרוזות לעמודות: לדוגמה, פיצול עמודה בשם `"Material_Sample"` ל־`"Material"` ו־`"Sample"`.

#### הסרת כפילויות במדידות (De-duplicating)

- `drop_duplicates()` — להסרת שורות של מדידות זהות שחוזרות על עצמן (למשל ניסוי שנמדד פעמיים באותו זמן).  
- ניתן לקבוע אם לשמור את ההופעה הראשונה, האחרונה, או להסיר את כל הכפילויות (`keep='first' | 'last' | False`).  

פעולה זו חיונית כדי להבטיח שהניתוחים הפיזיקליים והסטטיסטיים יתבססו על מדידות ייחודיות בלבד.

### תרגילים: ניקוי נתונים ניסיוניים, המרת יחידות והסרת כפילויות

לפני שנתחיל בתרגילים, ניצור טבלת נתונים קטנה שמדמה מצב מציאותי של **מדידות ניסוי “מלוכלכות”** —  
כאלה שנאספו ממכשירי מדידה שונים או מקבצי CSV גולמיים לפני עיבוד.  
הטבלה תכיל שגיאות אופייניות מעולם הפיזיקה הניסיונית, כגון:

- רווחים מיותרים בתחילת ובסוף מחרוזות (למשל בשמות חומרים או ניסויים).  
- ערכים לא אחידים בעמודת **נצילות (%)** — חלקם כוללים את סימן האחוז, חלקם ריקים או מסומנים כ־“n/a”.  
- עמודת **אנרגיה (J)** בפורמט טקסטואלי הכולל רווחים ואף מילים במקום מספרים.  
- שורות כפולות המייצגות **מדידות זהות** שחזרו על עצמן בניסוי.

לאחר יצירת הנתונים נבצע שלבי ניקוי מדורגים:
נחתוך רווחים, נפריד עמודות משולבות (כגון חומר ודגימה),  
נמיר יחידות או טיפוסים למספריים, נטפל בערכים חסרים,  
ולבסוף נסיר כפילויות — תהליך חיוני לפני כל **ניתוח פיזיקלי אמין**.


In [ ]:
import pandas as pd
import numpy as np

# --- Setup sample messy physics data ---
df = pd.DataFrame({
    "material_sample": ["  Aluminum A1", "Copper   B2  ", "  Steel  C3 ", "Aluminum A1", "PLASTIC   D4  "],
    "efficiency": ["10%", " 15% ", None, "n/a", "%20"],
    "energy_J": [" 100.0", "85.50 ", " thirty ", "120", " 200 "],
    "experiment": ["Exp1", "Exp1", " Exp2 ", "Exp1", "Exp2"]
})

print("Raw (messy) physics data:")
display(df)


#### ניקוי עמודות “מלוכלכות”: חיתוך רווחים ופיצול חומר+דגימה

נבצע:
1. חיתוך רווחים מיותרים `(str.strip)` בעמודות טקסט.
2. ננרמל רווחים כפולים לשם אחד `(str.replace(r'\s+', ' ', regex=True))`.
3. נפצל את `material_sample` ל־`material` (שם החומר) ו־`sample_id` (מזהה דגימה).

In [ ]:
# --- Clean whitespace and normalize spaces ---
# 1) Trim spaces around strings
df["material_sample"] = df["material_sample"].str.strip()
df["experiment"] = df["experiment"].str.strip()

# 2) Normalize multiple spaces to single space
df["material_sample"] = df["material_sample"].str.replace(r"\s+", " ", regex=True)

# 3) Split "Material SampleID" into two columns safely
split_cols = df["material_sample"].str.split(" ", n=1, expand=True)
df.loc[:, "material"] = split_cols[0].str.title()      # Title-case for consistency (Aluminum/Copper/Steel/Plastic)
df.loc[:, "sample_id"] = split_cols[1]

print("After trimming and splitting material and sample_id:")
display(df[["material_sample", "material", "sample_id", "experiment"]])


#### המרת אחוזי נצילות למספרים צפים ואימפוטציה לערכים חסרים

נמיר את עמודת `efficiency` מאחוזים במחרוזת לערך מספרי בין 0 ל־1:
- נסיר תווי `%` ורווחים.  
- נחליף ערכים לא תקינים (`'n/a'`, ריק) ל־NaN.  
- נהפוך ל־float ונמלא חסרים בערך ברירת מחדל (למשל 0).


In [ ]:
# --- Convert efficiency percentage strings to floats in [0, 1] and impute missing ---
eff = df["efficiency"].astype("string").str.strip()
eff = eff.str.replace("%", "", regex=False)            # remove percent sign
eff = pd.to_numeric(eff, errors="coerce")              # non-numeric -> NaN
eff = eff / 100.0                                      # scale to fraction
eff = eff.fillna(0.0)                                  # impute missing as 0 (no efficiency reported)

df.loc[:, "efficiency_frac"] = eff

print("Converted efficiency to fraction and imputed missing:")
display(df[["efficiency", "efficiency_frac"]])


#### המרת אנרגיה מטקסט למספרים וניקוי משתני ניסוי

- energy_J: נסיר רווחים ונהפוך למספר (pd.to_numeric, עם errors='coerce').
- experiment: חיתוך רווחים והמרה לקטגוריה (category) לחיסכון בזיכרון וקונסיסטנטיות.
- material: כבר תקנן־Title; מומלץ להמיר גם אותו לקטגוריה.

In [ ]:
# --- Clean energy and experiment/material types ---
# Energy to numeric (coerce invalid to NaN)
df["energy_J"] = pd.to_numeric(df["energy_J"].str.strip(), errors="coerce")

# Convert experiment and material to categorical types
df["experiment"] = df["experiment"].astype("category")
df["material"] = df["material"].astype("category")

print("Energy parsed and categorical types set:")
display(df[["energy_J", "experiment", "material"]])
print(df.dtypes)


#### הסרת שורות כפולות (Exact Duplicates)

נזהה ונשמיט כפילויות זהות לחלוטין בכל העמודות (או בתת־עמודות לבחירתנו).  
נשמור את ההופעה הראשונה (`keep='first'`).


In [ ]:
# --- Drop exact duplicate rows, keep first ---
before = len(df)
deduped = df.drop_duplicates(keep="first")
after = len(deduped)

print(f"Removed {before - after} duplicate rows (kept first occurrence).")
display(deduped)


`````{admonition} סיכום
:class: tip
- זיהינו וטיפלנו בערכים חסרים בעזרת המרה זהירה ואימפוטציה.  
- ניקינו מחרוזות עם `str.strip` ונרמלנו רווחים, כולל פיצול חכם של `material_sample` ל־`material` ו־`sample_id`.  
- המרנו אחוזי נצילות ממחרוזות ל־float (0–1) והשלמנו חסרים.  
- המרת טיפוסים: אנרגיה (`to_numeric`) וקטגוריות ניסוי/חומר (`astype('category')`).  
- הסרנו כפילויות מדויקות עם `drop_duplicates` כדי להבטיח ניתוח פיזיקלי אמין.
`````

In [10]:
import json
from jupyterquiz import display_quiz

quiz_json = '''
[
  {
    "question": "במהלך ניקוי נתוני ניסוי בפיזיקה נרצה להמיר עמודת <code>efficiency</code> שמכילה מחרוזות כמו '10%', ' 15% ', 'n/a' לערכים מספריים תקינים בין 0 ל־1, כדי שנוכל לבצע חישובים מדויקים של נצילות.<br><br>איזה רצף פעולות מתאים ביותר?",
    "type": "many_choice",
    "answers": [
      {
        "answer": "להסיר רווחים ו־% עם str.strip() ו־str.replace(), להמיר בעזרת pd.to_numeric(errors='coerce'), לחלק ב־100 ולמלא חסרים עם fillna(0)",
        "correct": true,
        "feedback": "נכון! זהו רצף הניקוי התקני בנתונים פיזיקליים: הסרת סימני אחוז, המרה למספרים, נירמול לטווח 0–1 והשלמת ערכים חסרים."
      },
      {
        "answer": "להשתמש ישירות ב־astype(float) על העמודה המקורית",
        "correct": false,
        "feedback": "לא מדויק — המרה ישירה תיכשל מאחר והעמודה מכילה תווים לא מספריים כמו '%' ו־'n/a'."
      },
      {
        "answer": "להשתמש ב־df.dropna() כדי למחוק את כל המדידות החסרות במקום לנקות אותן",
        "correct": false,
        "feedback": "לא נכון — השמטת שורות עלולה לגרום לאובדן נתונים; עדיף לנקות ולהשלים ערכים חסרים."
      },
      {
        "answer": "להמיר את הערכים ל־datetime כדי לבדוק זמני מדידה",
        "correct": false,
        "feedback": "לא רלוונטי — כאן מדובר בהמרת אחוזי נצילות, לא בתאריכים."
      }
    ]
  }
]
'''

myquiz = json.loads(quiz_json)
display_quiz(myquiz)


<IPython.core.display.Javascript object>